Student Name: Anh Ha & Phuong Nguyen

Download the full NSF Research Award Abstracts 1990-2003 Data Set

In [1]:
import numpy
import os
import re
import binascii
from time import time

def get_fnames():
    """Read all text files in a folder.
    """
    fnames = []
    for root,_,files in os.walk("NSF Research Award"):
        for fname in files:
            if fname[-4:] == ".txt":
                fnames.append(os.path.join(root, fname))
    return fnames

In [2]:
print("number of different files: {}".format(len(get_fnames())))

number of different files: 132372


In [3]:
# just open one file and read its abstract

def read_file(fname):
    with open(fname, 'r') as f:
        # skip all lines until abstract
        for line in f:
            if "Abstract    :" in line:
                break

        # get abstract as a single string
        abstract = ' '.join([line[:-1].strip() for line in f])
        abstract = re.sub(' +', ' ', abstract)  # remove double spaces
        return abstract

fname = get_fnames()[999]
print(fname,read_file(fname))

NSF Research Award\awards_1990\awd_1990_02\a9002673.txt Let L be an oriented link type in the oriented 3-sphere, S3, and let L and L', defined respectively by closed n- and m-braid diagrams, represent L. The investigator is primarily concerned with understanding how it is possible to jump between conjugacy classes without stabilization. This is one of the technical questions involved in understanding how a finite collection of (possibly knotted) simple closed curves in 3-dimensional space can be linked with one another.


Find the similar items using pair-wise comparisons, MinHash (vectorized version) and LSH. For each item describe your results. Compare the time and results for k=5 and 10 for the three methods and s=0.9.

In [4]:
def get_shingles(fname, k):
    """Get all shingles from requested file (hashes of these shingles)
    """
    with open(fname, 'r', errors = 'ignore') as f:
        # skip all lines until abstract
        for line in f:
            if "Abstract    :" in line:
                break

        # get abstract as a single string
        abstract = ' '.join([line[:-1].strip() for line in f])
        abstract = re.sub(' +', ' ', abstract)  # remove double spaces

        L = len(abstract)
        shingles = set()  # we use a set to automatically eliminate duplicates
        for i in range(L-k+1):
            shingle = abstract[i:i+k]
            crc = binascii.crc32(shingle.encode('utf-8')) #& 0xffffffff  # hash the shingle to a 32-bit integer
            shingles.add(crc)
        return shingles

In [5]:
fname = get_fnames()[999]
for i in [5,10]:
    print("file: {}".format(fname))
    print("number of shingles: {}".format(len(get_shingles(fname, i))))
    print("number of shingles: {}".format(get_shingles(fname, i)))

file: NSF Research Award\awards_1990\awd_1990_02\a9002673.txt
number of shingles: 425
number of shingles: {1046261761, 658997258, 3316230158, 2362679310, 3261571087, 1125644311, 4015761434, 436064291, 1188827171, 2860077100, 2085396524, 2130679863, 816592956, 4241432638, 3718907976, 3725809743, 4063817813, 2593511513, 3112093793, 450431081, 2734854251, 7460972, 2685941870, 503144566, 2183622783, 1537773700, 2333618322, 111411348, 3302146213, 3020867753, 3715932339, 1689852086, 2047129796, 2111555782, 2701545671, 2786472141, 3663186128, 2857177297, 2743720156, 2784258276, 2088812778, 528079083, 4079671540, 1136589046, 4197017850, 488622354, 2165571859, 1154173204, 2929092883, 4188360986, 1529047323, 1493747998, 3591567646, 1787549986, 76386605, 651956529, 83386674, 695386426, 1891262790, 2472841553, 601923926, 3926923609, 41511257, 3165954400, 3113120105, 681914731, 2893046132, 2929291643, 357226877, 2874313086, 3821087114, 1557731723, 3139531150, 2028259731, 134465943, 2759932311, 3000

In [6]:
def jaccard_similarity_score(x, y):
    intersection_cardinality = len(set(x).intersection(set(y)))
    union_cardinality = len(set(x).union(set(y)))
    return intersection_cardinality / float(union_cardinality)

In [7]:
fnames = get_fnames()[:1000]
shingles_vectors = []
for file in fnames:
    sh = list(get_shingles(file, k=5))
    shingles_vectors.append(sh)
print("Jaccard_similarity_score pair_wise k=5: {}".format(jaccard_similarity_score(shingles_vectors[0], shingles_vectors[1])))


Jaccard_similarity_score pair_wise k=5: 0.09054527263631816


In [8]:
fnames = get_fnames()[:1000]
shingles_vectors1 = []
for file in fnames:
    sh = list(get_shingles(file, k=10))
    shingles_vectors1.append(sh)
print("Jaccard_similarity_score pair_wise k=10: {}".format(jaccard_similarity_score(shingles_vectors1[0], shingles_vectors1[1])))

Jaccard_similarity_score pair_wise k=10: 0.013148283418553688


In [9]:
%%time
import itertools

s = 0.9
fnames = get_fnames()[:1000]

pair_wise = []
for pair in itertools.combinations(fnames,2):
    js = jaccard_similarity_score(get_shingles(pair[0], k=5),get_shingles(pair[1], k=5))

    if js > s:
        print(pair)
        pair_wise.append(pair)
print("Number of similar items k=5: {}".format(len(pair_wise)))

('NSF Research Award\\awards_1990\\awd_1990_00\\a9000177.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000458.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000221.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000222.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000221.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000223.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000221.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000396.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000221.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000404.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000221.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000528.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000221.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000944.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000221.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001055.txt')


('NSF Research Award\\awards_1990\\awd_1990_00\\a9000223.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000528.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000223.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000944.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000223.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001055.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000223.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001222.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000223.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001417.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000223.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001497.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000223.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000223.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001612.txt')


('NSF Research Award\\awards_1990\\awd_1990_00\\a9000396.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002452.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000396.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002460.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000396.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002519.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000396.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002570.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000404.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000528.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000404.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000944.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000404.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001055.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000404.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001222.txt')


('NSF Research Award\\awards_1990\\awd_1990_00\\a9000528.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002519.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000528.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002570.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000944.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001055.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000944.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001222.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000944.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001417.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000944.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001497.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000944.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000944.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001612.txt')


('NSF Research Award\\awards_1990\\awd_1990_01\\a9001217.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001218.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001217.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001582.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001217.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002509.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001218.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001582.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001218.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002509.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001222.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001417.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001222.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001497.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001222.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt')


('NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001612.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001837.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001846.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001847.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001914.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001947.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001980.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002066.txt')


('NSF Research Award\\awards_1990\\awd_1990_01\\a9001846.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002452.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001846.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002460.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001846.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002519.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001846.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002570.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001847.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001914.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001847.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001947.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001847.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001980.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001847.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002066.txt')


('NSF Research Award\\awards_1990\\awd_1990_02\\a9002218.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002269.txt')
('NSF Research Award\\awards_1990\\awd_1990_02\\a9002218.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002332.txt')
('NSF Research Award\\awards_1990\\awd_1990_02\\a9002218.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002452.txt')
('NSF Research Award\\awards_1990\\awd_1990_02\\a9002218.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002460.txt')
('NSF Research Award\\awards_1990\\awd_1990_02\\a9002218.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002519.txt')
('NSF Research Award\\awards_1990\\awd_1990_02\\a9002218.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002570.txt')
('NSF Research Award\\awards_1990\\awd_1990_02\\a9002269.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002332.txt')
('NSF Research Award\\awards_1990\\awd_1990_02\\a9002269.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002452.txt')


In [10]:
%%time
import itertools

s = 0.9
fnames = get_fnames()[:1000]

pair_wise1 = []
for pair in itertools.combinations(fnames,2):
    js = jaccard_similarity_score(get_shingles(pair[0], k=10),get_shingles(pair[1], k=10))

    if js > s:
        print(pair)
        pair_wise1.append(pair)
print("Number of similar items k=10: {}".format(len(pair_wise1)))


('NSF Research Award\\awards_1990\\awd_1990_00\\a9000177.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000458.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000221.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000222.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000221.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000223.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000221.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000396.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000221.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000404.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000221.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000528.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000221.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000944.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000221.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001055.txt')


('NSF Research Award\\awards_1990\\awd_1990_00\\a9000223.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000528.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000223.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000944.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000223.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001055.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000223.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001222.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000223.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001417.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000223.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001497.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000223.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000223.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001612.txt')


('NSF Research Award\\awards_1990\\awd_1990_00\\a9000396.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002452.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000396.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002460.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000396.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002519.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000396.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002570.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000404.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000528.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000404.txt', 'NSF Research Award\\awards_1990\\awd_1990_00\\a9000944.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000404.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001055.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000404.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001222.txt')


('NSF Research Award\\awards_1990\\awd_1990_00\\a9000528.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002519.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000528.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002570.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000944.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001055.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000944.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001222.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000944.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001417.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000944.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001497.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000944.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt')
('NSF Research Award\\awards_1990\\awd_1990_00\\a9000944.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001612.txt')


('NSF Research Award\\awards_1990\\awd_1990_01\\a9001217.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001218.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001217.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001582.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001217.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002509.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001218.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001582.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001218.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002509.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001222.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001417.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001222.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001497.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001222.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt')


('NSF Research Award\\awards_1990\\awd_1990_01\\a9001497.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002519.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001497.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002570.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001612.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001837.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001846.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001847.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001914.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001558.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001947.txt')


('NSF Research Award\\awards_1990\\awd_1990_01\\a9001846.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002452.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001846.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002460.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001846.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002519.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001846.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002570.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001847.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001914.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001847.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001947.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001847.txt', 'NSF Research Award\\awards_1990\\awd_1990_01\\a9001980.txt')
('NSF Research Award\\awards_1990\\awd_1990_01\\a9001847.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002066.txt')


('NSF Research Award\\awards_1990\\awd_1990_02\\a9002218.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002269.txt')
('NSF Research Award\\awards_1990\\awd_1990_02\\a9002218.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002332.txt')
('NSF Research Award\\awards_1990\\awd_1990_02\\a9002218.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002452.txt')
('NSF Research Award\\awards_1990\\awd_1990_02\\a9002218.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002460.txt')
('NSF Research Award\\awards_1990\\awd_1990_02\\a9002218.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002519.txt')
('NSF Research Award\\awards_1990\\awd_1990_02\\a9002218.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002570.txt')
('NSF Research Award\\awards_1990\\awd_1990_02\\a9002269.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002332.txt')
('NSF Research Award\\awards_1990\\awd_1990_02\\a9002269.txt', 'NSF Research Award\\awards_1990\\awd_1990_02\\a9002452.txt')


Minhash_vectorized method

In [11]:
# set global parameters to process the whole dataset
bands = 5
rows = 5
nsig = bands*rows  # number of elements in signature, or the number of different random hash functions

maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
nextPrime = 4294967311  # next prime number after maxShingleID

A = numpy.random.randint(0, nextPrime/2, size=(nsig,), dtype=numpy.int64)
B = numpy.random.randint(0, nextPrime/2, size=(nsig,), dtype=numpy.int64)



In [12]:
print(len(A),A)

25 [1141323798  633531676  367712933 1424617046  311088446 1268720329
 1460876892 1102172141  467763116  766573562  399553721 1418774302
 2044329893  843080240  702275610  544967034 1183207659 1646604486
 2091868304 1182256917 1918254418 2054617894  242686343  549607344
   53650881]


In [13]:
def minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig):
    signature = numpy.ones((nsig,)) * (maxShingleID + 1)

    for ShingleID in shingles:
        hashCodes = ((A*ShingleID + B) % nextPrime) % maxShingleID
        numpy.minimum(signature, hashCodes, out=signature)

    return signature

In [14]:
signatures = []  # signatures for all files
fnames = get_fnames()[:1000]
for fname in fnames:
    shingles = list(get_shingles(fname, k=5))
    sig = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
    signatures.append(sig)
print("Jaccard_similarity_score: {}".format(jaccard_similarity_score(signatures[0], signatures[1])))
s = 0.9  # similarity threshold
Nfiles = len(signatures)
t = time()
minhash = []
for i in range(Nfiles):
    for j in range(i+1, Nfiles):
        Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in two vectors
        if Jsim >= s:
            minhash.append((i,j))
t2 = time() - t
print("finding minhash similar pairs k=5 s=0.9 took {} seconds".format(t2))
print("found {} minhash similar pairs k=5 s=0.9 ".format(len(minhash)))
print("minhash similar pairs k=5 s=0.9 of files are:")
for i,j in minhash:
    print(fnames[i], fnames[j])


Jaccard_similarity_score: 0.041666666666666664
finding minhash similar pairs k=5 s=0.9 took 5.961978435516357 seconds
found 576 minhash similar pairs k=5 s=0.9 
minhash similar pairs k=5 s=0.9 of files are:
NSF Research Award\awards_1990\awd_1990_00\a9000046.txt NSF Research Award\awards_1990\awd_1990_00\a9000158.txt
NSF Research Award\awards_1990\awd_1990_00\a9000177.txt NSF Research Award\awards_1990\awd_1990_00\a9000458.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000222.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000223.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000396.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000404.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000528.txt
NSF Resea

In [15]:
signatures1 = []  # signatures for all files
fnames = get_fnames()[:1000]
for fname in fnames:
    shingles1 = get_shingles(fname, k=10)
    sig1 = minhash_vectorized(shingles1, A, B, nextPrime, maxShingleID, nsig)
    signatures1.append(sig1)
    
print("Jaccard_similarity_score: {}".format(jaccard_similarity_score(signatures1[0], signatures1[1])))
s = 0.9  # similarity threshold
Nfiles1 = len(signatures1)
t3 = time()
minhash1 = []
for i in range(Nfiles1):
    for j in range(i+1, Nfiles1):
        Jsim1 = numpy.mean(signatures1[i] == signatures1[j])  # average number of similar items in two vectors
        if Jsim1 >= s:
            minhash1.append((i,j))
t4 = time() - t3
print("finding minhash similar pairs k=10 s=0.9 took {} seconds".format(t4))
print("found {} minhash similar pairs k=10 s=0.9 ".format(len(minhash1)))
print("minhash similar pairs k=10 s=0.9 of files are:")
for i,j in minhash1:
    print(fnames[i], fnames[j])


Jaccard_similarity_score: 0.02040816326530612
finding minhash similar pairs k=10 s=0.9 took 5.965031862258911 seconds
found 577 minhash similar pairs k=10 s=0.9 
minhash similar pairs k=10 s=0.9 of files are:
NSF Research Award\awards_1990\awd_1990_00\a9000177.txt NSF Research Award\awards_1990\awd_1990_00\a9000458.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000222.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000223.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000396.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000404.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000528.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000944.txt
NSF Res

LSH method

In [16]:
def LSH(signatures, bands, rows, Ab, Bb, nextPrime, maxShingleID):
   
    numItems = signatures.shape[1]
    signBands = numpy.array_split(signatures, bands, axis=0)
    candidates = set()
    for nb in range(bands):
        hashTable = {}
        for ni in range(numItems):
            item = signBands[nb][:,ni]
            hash = (numpy.dot(Ab[nb,:], item) + Bb[nb]) % nextPrime % maxShingleID
            if hash not in hashTable:
                hashTable[hash] = [ni]
            else:
                hashTable[hash].append(ni)
        for _,items in hashTable.items():
            if len(items) > 1:
                L = len(items)
                for i in range(L-1):
                    for j in range(i+1, L):
                        cand = [items[i], items[j]]
                        numpy.sort(cand)
                        candidates.add(tuple(cand))
    return candidates

In [17]:
# find candidates with LSH
signatures2 = []  # signatures for all files
fnames = get_fnames()[:1000]
for fname in fnames:
    shingles2 = get_shingles(fname, k=5)
    sig2 = minhash_vectorized(shingles2, A, B, nextPrime, maxShingleID, nsig)
    signatures2.append(sig2)

# prepare data for LSH
A2 = numpy.random.randint(0, nextPrime/2, size=(bands, rows), dtype=numpy.int64)  # now we need a vector of A parameters for each band
B2 = numpy.random.randint(0, nextPrime/2, size=(bands, ), dtype=numpy.int64)
signatures2 = numpy.array(signatures2).T  # LSH needs a matrix of signatures, not a list of vectors

s = 0.9 # similarity threshold
Nfiles2 = signatures2.shape[1]  # number of different files
t5 = time()
candidates = LSH(signatures2, bands, rows, A2, B2, nextPrime, maxShingleID)
t6 = time() - t5
print("finding candidates k=5 s=0.9 took {} seconds".format(t6))
print("found {} candidates k=5 s=0.9 ".format(len(candidates)))
print("candidate similar pairs k=5 s=0.9 of files are:")
for i,j in candidates:
    print(fnames[i], fnames[j])


finding candidates k=5 s=0.9 took 0.047614336013793945 seconds
found 617 candidates k=5 s=0.9 
candidate similar pairs k=5 s=0.9 of files are:
NSF Research Award\awards_1990\awd_1990_00\a9000962.txt NSF Research Award\awards_1990\awd_1990_01\a9001029.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000222.txt
NSF Research Award\awards_1990\awd_1990_00\a9000222.txt NSF Research Award\awards_1990\awd_1990_02\a9002519.txt
NSF Research Award\awards_1990\awd_1990_00\a9000223.txt NSF Research Award\awards_1990\awd_1990_02\a9002452.txt
NSF Research Award\awards_1990\awd_1990_01\a9001417.txt NSF Research Award\awards_1990\awd_1990_02\a9002460.txt
NSF Research Award\awards_1990\awd_1990_02\a9002178.txt NSF Research Award\awards_1990\awd_1990_02\a9002519.txt
NSF Research Award\awards_1990\awd_1990_02\a9002127.txt NSF Research Award\awards_1990\awd_1990_02\a9002176.txt
NSF Research Award\awards_1990\awd_1990_02\a9002179.txt NSF Research Awar

In [18]:
signatures3 = []  # signatures for all files
fnames = get_fnames()[:1000]
for fname in fnames:
    shingles3 = get_shingles(fname, k=10)
    sig3 = minhash_vectorized(shingles3, A, B, nextPrime, maxShingleID, nsig)
    signatures3.append(sig3)

# prepare data for LSH
A2 = numpy.random.randint(0, nextPrime/2, size=(bands, rows), dtype=numpy.int64)  # now we need a vector of A parameters for each band
B2 = numpy.random.randint(0, nextPrime/2, size=(bands, ), dtype=numpy.int64)
signatures3 = numpy.array(signatures).T  # LSH needs a matrix of signatures, not a list of vectors

s = 0.9 # similarity threshold
Nfiles3 = signatures3.shape[1]  # number of different files
t7 = time()
candidates1 = LSH(signatures3, bands, rows, A2, B2, nextPrime, maxShingleID)
t8 = time() - t7
print("finding candidates k=10 s=0.9 took {} seconds".format(t8))
print("found {} candidates k=10 s=0.9 ".format(len(candidates1)))
print("candidate similar pairs k=10 s=0.9 of files are:")
for i,j in candidates1:
    print(fnames[i], fnames[j])



finding candidates k=10 s=0.9 took 0.0379786491394043 seconds
found 617 candidates k=10 s=0.9 
candidate similar pairs k=10 s=0.9 of files are:
NSF Research Award\awards_1990\awd_1990_00\a9000962.txt NSF Research Award\awards_1990\awd_1990_01\a9001029.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000222.txt
NSF Research Award\awards_1990\awd_1990_00\a9000222.txt NSF Research Award\awards_1990\awd_1990_02\a9002519.txt
NSF Research Award\awards_1990\awd_1990_00\a9000223.txt NSF Research Award\awards_1990\awd_1990_02\a9002452.txt
NSF Research Award\awards_1990\awd_1990_01\a9001417.txt NSF Research Award\awards_1990\awd_1990_02\a9002460.txt
NSF Research Award\awards_1990\awd_1990_02\a9002178.txt NSF Research Award\awards_1990\awd_1990_02\a9002519.txt
NSF Research Award\awards_1990\awd_1990_02\a9002127.txt NSF Research Award\awards_1990\awd_1990_02\a9002176.txt
NSF Research Award\awards_1990\awd_1990_02\a9002179.txt NSF Research Awa

In [40]:
print("Jaccard_similarity_score pair_wise k=5: {}".format(jaccard_similarity_score(shingles_vectors[0], shingles_vectors[1])))
print("Jaccard_similarity_score pair_wise k=10: {}".format(jaccard_similarity_score(shingles_vectors1[0], shingles_vectors1[1])))
print("Jaccard_similarity_score Minhash_vectorizer k=5: {}".format(jaccard_similarity_score(signatures[0], signatures[1])))
print("Jaccard_similarity_score  Minhash_vectorizer k=10: {}".format(jaccard_similarity_score(signatures1[0], signatures1[1])))

Jaccard_similarity_score pair_wise k=5: 0.09054527263631816
Jaccard_similarity_score pair_wise k=10: 0.013148283418553688
Jaccard_similarity_score Minhash_vectorizer k=5: 0.041666666666666664
Jaccard_similarity_score  Minhash_vectorizer k=10: 0.02040816326530612


In [41]:
print("Number of similar items k=5: {}".format(len(pair_wise)))
print("Number of similar items k=10: {}".format(len(pair_wise1)))
print("found {} minhash similar pairs k=5 s=0.9 ".format(len(minhash)))
print("found {} minhash similar pairs k=10 s=0.9 ".format(len(minhash1)))
print("found {} LSH candidates k=5 s=0.9 ".format(len(candidates)))
print("found {} LSH candidates k=10 s=0.9 ".format(len(candidates1)))

Number of similar items k=5: 575
Number of similar items k=10: 575
found 576 minhash similar pairs k=5 s=0.9 
found 577 minhash similar pairs k=10 s=0.9 
found 617 LSH candidates k=5 s=0.9 
found 617 LSH candidates k=10 s=0.9 


In [42]:
print("finding minhash similar pairs k=5 s=0.9 took {} seconds".format(t2))
print("finding minhash similar pairs k=10 s=0.9 took {} seconds".format(t4))
print("finding LSH candidates k=5 s=0.9 took {} seconds".format(t6))
print("finding LSH candidates k=10 s=0.9 took {} seconds".format(t8))

finding minhash similar pairs k=5 s=0.9 took 5.961978435516357 seconds
finding minhash similar pairs k=10 s=0.9 took 5.965031862258911 seconds
finding LSH candidates k=5 s=0.9 took 0.047614336013793945 seconds
finding LSH candidates k=10 s=0.9 took 0.0379786491394043 seconds


Conclusion: 
1. Jaccard_similarity_score for k=10 is smaller than k=5
2. The numer of similarity pairs is nearly same in case of k=5 or 10
3. Time for k=10 in LSH method is less than for k=5, but for Minhash_vectorizer time is the same for k=5 or 10


Compare the results for different similarity thresholds (s = 0.5, 0.9 and 0.95) for MinHash and LSH (10, 100 and 200 hashing functions).

MinHash_vectorized s = 0.5, 0.9, 0.95

In [24]:
signatures4 = []  # signatures for all files
fnames = get_fnames()[:1000]
for fname in fnames:
    shingles2 = list(get_shingles(fname, k=5))
    sig4 = minhash_vectorized(shingles2, A, B, nextPrime, maxShingleID, nsig)
    signatures4.append(sig4)
print("Jaccard_similarity_score: {}".format(jaccard_similarity_score(signatures4[0], signatures4[1])))
s = 0.5  # similarity threshold
Nfiles4 = len(signatures4)
t9 = time()
minhash2 = []
for i in range(Nfiles4):
    for j in range(i+1, Nfiles4):
        Jsim2 = numpy.mean(signatures4[i] == signatures4[j])  # average number of similar items in two vectors
        if Jsim2 >= s:
            minhash2.append((i,j))
t10 = time() - t9
print("finding candidates k=5 s=0.5 took {} seconds".format(t10))
print("found {} candidates k=5 s=0.5 ".format(len(minhash2)))
print("candidate similar pairs k=5 s=0.5 of files are:")
for i,j in minhash2:
    print(fnames[i], fnames[j])

Jaccard_similarity_score: 0.041666666666666664
finding candidates k=5 s=0.5 took 6.456842660903931 seconds
found 689 candidates k=5 s=0.5 
candidate similar pairs k=5 s=0.5 of files are:
NSF Research Award\awards_1990\awd_1990_00\a9000046.txt NSF Research Award\awards_1990\awd_1990_00\a9000049.txt
NSF Research Award\awards_1990\awd_1990_00\a9000046.txt NSF Research Award\awards_1990\awd_1990_00\a9000130.txt
NSF Research Award\awards_1990\awd_1990_00\a9000046.txt NSF Research Award\awards_1990\awd_1990_00\a9000158.txt
NSF Research Award\awards_1990\awd_1990_00\a9000046.txt NSF Research Award\awards_1990\awd_1990_00\a9000246.txt
NSF Research Award\awards_1990\awd_1990_00\a9000046.txt NSF Research Award\awards_1990\awd_1990_00\a9000251.txt
NSF Research Award\awards_1990\awd_1990_00\a9000046.txt NSF Research Award\awards_1990\awd_1990_00\a9000343.txt
NSF Research Award\awards_1990\awd_1990_00\a9000046.txt NSF Research Award\awards_1990\awd_1990_00\a9000393.txt
NSF Research Award\awards_199

In [25]:
signatures5 = []  # signatures for all files
fnames = get_fnames()[:1000]
for fname in fnames:
    shingles3 = list(get_shingles(fname, k=5))
    sig5 = minhash_vectorized(shingles3, A, B, nextPrime, maxShingleID, nsig)
    signatures5.append(sig5)
print("Jaccard_similarity_score: {}".format(jaccard_similarity_score(signatures5[0], signatures5[1])))
s = 0.95  # similarity threshold
Nfiles5 = len(signatures5)
t11 = time()
minhash3 = []
for i in range(Nfiles5):
    for j in range(i+1, Nfiles5):
        Jsim3 = numpy.mean(signatures5[i] == signatures5[j])  # average number of similar items in two vectors
        if Jsim3 >= s:
            minhash3.append((i,j))
t12 = time() - t11
print("finding candidates k=5 s=0.95 took {} seconds".format(t12))
print("found {} candidates k=5 s=0.95 ".format(len(minhash3)))
print("candidate similar pairs k=5 s=0.95 of files are:")
for i,j in minhash3:
    print(fnames[i], fnames[j])

Jaccard_similarity_score: 0.041666666666666664
finding candidates k=5 s=0.95 took 7.316863775253296 seconds
found 575 candidates k=5 s=0.95 
candidate similar pairs k=5 s=0.95 of files are:
NSF Research Award\awards_1990\awd_1990_00\a9000177.txt NSF Research Award\awards_1990\awd_1990_00\a9000458.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000222.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000223.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000396.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000404.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000528.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000944.txt
NSF Research Award\awards_

LSH 10, 100, 200 hashing functions

In [26]:
bands1 = 2
rows1 = 5
nsig1 = bands1*rows1  # number of elements in signature, or the number of different random hash functions

maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
nextPrime = 4294967311  # next prime number after maxShingleID

A1 = numpy.random.randint(0, nextPrime/2, size=(nsig1,), dtype=numpy.int64)
B1 = numpy.random.randint(0, nextPrime/2, size=(nsig1,), dtype=numpy.int64)


In [27]:
signatures6 = []  # signatures for all files
fnames = get_fnames()[:1000]
for fname in fnames:
    shingles4 = get_shingles(fname, k=5)
    sig6 = minhash_vectorized(shingles4, A1, B1, nextPrime, maxShingleID, nsig1)
    signatures6.append(sig6)

# prepare data for LSH
A3 = numpy.random.randint(0, nextPrime/2, size=(bands1, rows1), dtype=numpy.int64)  # now we need a vector of A parameters for each band
B3 = numpy.random.randint(0, nextPrime/2, size=(bands1, ), dtype=numpy.int64)
signatures6 = numpy.array(signatures6).T  # LSH needs a matrix of signatures, not a list of vectors

s = 0.9 # similarity threshold
Nfiles6 = signatures6.shape[1]  # number of different files
t13 = time()
candidates2 = LSH(signatures6, bands1, rows1, A3, B3, nextPrime, maxShingleID)
t14 = time() - t13
print("finding candidates 10 hash functions took {} seconds".format(t14))
print("found {} candidates 10 hash functions ".format(len(candidates2)))
print("candidate similar pairs 10 hash functions of files are:")
for i,j in candidates2:
    print(fnames[i], fnames[j])


finding candidates 10 hash functions took 0.014989852905273438 seconds
found 591 candidates 10 hash functions 
candidate similar pairs 10 hash functions of files are:
NSF Research Award\awards_1990\awd_1990_00\a9000962.txt NSF Research Award\awards_1990\awd_1990_01\a9001029.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000222.txt
NSF Research Award\awards_1990\awd_1990_00\a9000222.txt NSF Research Award\awards_1990\awd_1990_02\a9002519.txt
NSF Research Award\awards_1990\awd_1990_00\a9000223.txt NSF Research Award\awards_1990\awd_1990_02\a9002452.txt
NSF Research Award\awards_1990\awd_1990_01\a9001417.txt NSF Research Award\awards_1990\awd_1990_02\a9002460.txt
NSF Research Award\awards_1990\awd_1990_02\a9002178.txt NSF Research Award\awards_1990\awd_1990_02\a9002519.txt
NSF Research Award\awards_1990\awd_1990_02\a9002127.txt NSF Research Award\awards_1990\awd_1990_02\a9002176.txt
NSF Research Award\awards_1990\awd_1990_02\a90021

In [28]:
bands2 = 20
rows2 = 5
nsig2 = bands2*rows2  # number of elements in signature, or the number of different random hash functions

maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
nextPrime = 4294967311  # next prime number after maxShingleID

A4 = numpy.random.randint(0, nextPrime/2, size=(nsig2,), dtype=numpy.int64)
B4 = numpy.random.randint(0, nextPrime/2, size=(nsig2,), dtype=numpy.int64)

In [29]:
signatures7 = []  # signatures for all files
fnames = get_fnames()[:1000]
for fname in fnames:
    shingles5 = get_shingles(fname, k=5)
    sig7 = minhash_vectorized(shingles5, A4, B4, nextPrime, maxShingleID, nsig2)
    signatures7.append(sig7)

# prepare data for LSH
A6 = numpy.random.randint(0, nextPrime/2, size=(bands2, rows2), dtype=numpy.int64)  # now we need a vector of A parameters for each band
B6 = numpy.random.randint(0, nextPrime/2, size=(bands2, ), dtype=numpy.int64)
signatures7 = numpy.array(signatures7).T  # LSH needs a matrix of signatures, not a list of vectors

s = 0.9 # similarity threshold
Nfiles7 = signatures7.shape[1]  # number of different files
t15 = time()
candidates3 = LSH(signatures7, bands2, rows2, A6, B6, nextPrime, maxShingleID)
t16 = time() - t15
print("finding candidates 100 hash function took {} seconds".format(t16))
print("found {} candidates 100 hash function ".format(len(candidates3)))
print("candidate similar pairs 100 hash function of files are:")
for i,j in candidates3:
    print(fnames[i], fnames[j])


finding candidates 100 hash function took 0.1723041534423828 seconds
found 713 candidates 100 hash function 
candidate similar pairs 100 hash function of files are:
NSF Research Award\awards_1990\awd_1990_01\a9001485.txt NSF Research Award\awards_1990\awd_1990_02\a9002351.txt
NSF Research Award\awards_1990\awd_1990_00\a9000962.txt NSF Research Award\awards_1990\awd_1990_01\a9001029.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000222.txt
NSF Research Award\awards_1990\awd_1990_00\a9000222.txt NSF Research Award\awards_1990\awd_1990_02\a9002519.txt
NSF Research Award\awards_1990\awd_1990_00\a9000223.txt NSF Research Award\awards_1990\awd_1990_02\a9002452.txt
NSF Research Award\awards_1990\awd_1990_01\a9001417.txt NSF Research Award\awards_1990\awd_1990_02\a9002460.txt
NSF Research Award\awards_1990\awd_1990_02\a9002178.txt NSF Research Award\awards_1990\awd_1990_02\a9002519.txt
NSF Research Award\awards_1990\awd_1990_02\a9002127

In [30]:
bands3 = 20
rows3 = 10
nsig3 = bands3*rows3  # number of elements in signature, or the number of different random hash functions

maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
nextPrime = 4294967311  # next prime number after maxShingleID

A5 = numpy.random.randint(0, nextPrime/2, size=(nsig3,), dtype=numpy.int64)
B5 = numpy.random.randint(0, nextPrime/2, size=(nsig3,), dtype=numpy.int64)

In [31]:
signatures8 = []  # signatures for all files
fnames = get_fnames()[:1000]
for fname in fnames:
    shingles6 = get_shingles(fname, k=5)
    sig8 = minhash_vectorized(shingles6, A5, B5, nextPrime, maxShingleID, nsig3)
    signatures8.append(sig8)

# prepare data for LSH
A7 = numpy.random.randint(0, nextPrime/2, size=(bands3, rows3), dtype=numpy.int64)  # now we need a vector of A parameters for each band
B7 = numpy.random.randint(0, nextPrime/2, size=(bands3, ), dtype=numpy.int64)
signatures8 = numpy.array(signatures8).T  # LSH needs a matrix of signatures, not a list of vectors

s = 0.9 # similarity threshold
Nfiles8 = signatures8.shape[1]  # number of different files
t17 = time()
candidates4 = LSH(signatures8, bands3, rows3, A7, B7, nextPrime, maxShingleID)
t18 = time() - t17
print("finding candidates 200 hash function took {} seconds".format(t18))
print("found {} candidates 200 hash function ".format(len(candidates4)))
print("candidate similar pairs 200 hash function of files are:")
for i,j in candidates4:
    print(fnames[i], fnames[j])


finding candidates 200 hash function took 0.19128870964050293 seconds
found 594 candidates 200 hash function 
candidate similar pairs 200 hash function of files are:
NSF Research Award\awards_1990\awd_1990_00\a9000962.txt NSF Research Award\awards_1990\awd_1990_01\a9001029.txt
NSF Research Award\awards_1990\awd_1990_00\a9000221.txt NSF Research Award\awards_1990\awd_1990_00\a9000222.txt
NSF Research Award\awards_1990\awd_1990_00\a9000222.txt NSF Research Award\awards_1990\awd_1990_02\a9002519.txt
NSF Research Award\awards_1990\awd_1990_00\a9000223.txt NSF Research Award\awards_1990\awd_1990_02\a9002452.txt
NSF Research Award\awards_1990\awd_1990_01\a9001417.txt NSF Research Award\awards_1990\awd_1990_02\a9002460.txt
NSF Research Award\awards_1990\awd_1990_02\a9002178.txt NSF Research Award\awards_1990\awd_1990_02\a9002519.txt
NSF Research Award\awards_1990\awd_1990_02\a9002127.txt NSF Research Award\awards_1990\awd_1990_02\a9002176.txt
NSF Research Award\awards_1990\awd_1990_02\a900217

In [43]:
print("Jaccard_similarity_score Minhash_vectorizer k=5 s=0.5: {}".format(jaccard_similarity_score(signatures4[0], signatures4[1])))
print("Jaccard_similarity_score Minhash_vectorizer k=5 s=0.9: {}".format(jaccard_similarity_score(signatures[0], signatures[1])))
print("Jaccard_similarity_score Minhash_vectorizer k=5 s=0.95: {}".format(jaccard_similarity_score(signatures5[0], signatures5[1])))

Jaccard_similarity_score Minhash_vectorizer k=5 s=0.5: 0.041666666666666664
Jaccard_similarity_score Minhash_vectorizer k=5 s=0.9: 0.041666666666666664
Jaccard_similarity_score Minhash_vectorizer k=5 s=0.95: 0.041666666666666664


In [44]:
print("finding minhash_vectorizer k=5 s=0.5 took {} seconds".format(t10))
print("finding minhash_vectorizer k=5 s=0.9 took {} seconds".format(t2))
print("finding minhash_vectorizer k=5 s=0.95 took {} seconds".format(t12))
print("finding LSH candidates 10 hash functions took {} seconds".format(t14))
print("finding LSH candidates 100 hash function took {} seconds".format(t16))
print("finding LSH candidates 200 hash function took {} seconds".format(t18))


finding minhash_vectorizer k=5 s=0.5 took 6.456842660903931 seconds
finding minhash_vectorizer k=5 s=0.9 took 5.961978435516357 seconds
finding minhash_vectorizer k=5 s=0.95 took 7.316863775253296 seconds
finding LSH candidates 10 hash functions took 0.014989852905273438 seconds
finding LSH candidates 100 hash function took 0.1723041534423828 seconds
finding LSH candidates 200 hash function took 0.19128870964050293 seconds


In [46]:
print("found {} minhash_vectorizer k=5 s=0.5 ".format(len(minhash2)))
print("found {} minhash similar pairs k=5 s=0.9 ".format(len(minhash)))
print("found {} minhash_vectorizer k=5 s=0.95 ".format(len(minhash3)))
print("found {} LSH candidates 10 hash function ".format(len(candidates2)))
print("found {} LSH candidates 100 hash function ".format(len(candidates3)))
print("found {} LSH candidates 200 hash function ".format(len(candidates4)))

found 689 minhash_vectorizer k=5 s=0.5 
found 576 minhash similar pairs k=5 s=0.9 
found 575 minhash_vectorizer k=5 s=0.95 
found 591 LSH candidates 10 hash function 
found 713 LSH candidates 100 hash function 
found 594 LSH candidates 200 hash function 


Comparition the results
1. Time for LSH with higher hash function is longer than smaller hash function
2. The similarity pairs for Minhash_vectorizer with smaller s are more than with higher s
3. The similarity pairs for LSH with 100 hash function are found more than with 10 or 200 hash function. It is neccesary of finding the optimal hash function. 

Using a 1-NN criteria, identify the clusters of similar abstracts for each method and s. Are the cluster different?, Upon inspection, what is the topic of each cluster? Check for example https://www.coursera.org/lecture/ml-clustering-and-retrieval/1-nn-algorithm-6wIOD

In [35]:
print(type(candidates3))

<class 'set'>


In [36]:
print(len(candidates3))

713


In [37]:
l = list(candidates3)

result = []
if len(l) > 1:
  tmp = [l[0]]
  for i in range(1,len(l)):
    if l[i][0] == l[i-1][1] or l[i][1] == l[i-1][0] or l[i][1] == l[i-1][1] or l[i][0] == l[i-1][0]:
      tmp.append(l[i])
    else:
      result.append(tmp)
      tmp = [l[i]]
  result.append(tmp)
else:
  result = l

for elem in result:
  print(elem)

[(564, 865)]
[(365, 390)]
[(67, 68), (68, 939)]
[(69, 904)]
[(540, 907)]
[(812, 939)]
[(786, 810)]
[(813, 904)]
[(695, 939)]
[(47, 71)]
[(587, 857)]
[(463, 571), (67, 571)]
[(463, 809)]
[(68, 786)]
[(698, 810)]
[(132, 133)]
[(695, 786), (587, 695)]
[(127, 463)]
[(786, 904)]
[(812, 813)]
[(69, 778)]
[(129, 402)]
[(695, 813)]
[(587, 960)]
[(179, 766)]
[(587, 731)]
[(697, 841)]
[(721, 960)]
[(810, 907)]
[(67, 463)]
[(68, 907)]
[(102, 155)]
[(463, 939)]
[(746, 980)]
[(695, 907)]
[(587, 825)]
[(71, 442)]
[(247, 259)]
[(721, 825)]
[(463, 786)]
[(106, 390)]
[(540, 731)]
[(276, 331)]
[(463, 813), (67, 813)]
[(280, 292)]
[(810, 857)]
[(209, 259)]
[(766, 841), (741, 766)]
[(910, 962)]
[(540, 605)]
[(179, 361)]
[(818, 982)]
[(94, 155)]
[(303, 387)]
[(115, 861)]
[(361, 904)]
[(127, 179)]
[(571, 741)]
[(390, 391)]
[(196, 962)]
[(750, 981)]
[(766, 809)]
[(69, 741)]
[(6, 47)]
[(825, 904)]
[(331, 332)]
[(68, 129)]
[(605, 721)]
[(698, 809)]
[(361, 778), (778, 810)]
[(67, 179)]
[(841, 857)]
[(8, 102)]
[

Implement a cosine distance and compare with a Jacard distance for the vectorized MinHash algorithm. How they compare in performance? Results? 


In [38]:
dis_sig = []  # signatures for all files
fnames = get_fnames()[:1000]
for fname in fnames:
    shingles7 = list(get_shingles(fname, k=5))
    sig9 = minhash_vectorized(shingles7, A4, B4, nextPrime, maxShingleID, nsig2)
    dis_sig.append(sig9)
a, b = dis_sig[0], dis_sig[1]

print("Jaccard_similarity_score: {}".format(jaccard_similarity_score(a,b))) 
print("Jaccard_distance: {}".format(1-jaccard_similarity_score(a,b))) 
from scipy import spatial
print("Cosine_distance: {}".format(spatial.distance.cosine(a,b)))
    

Jaccard_similarity_score: 0.03626943005181347
Jaccard_distance: 0.9637305699481865
Cosine_distance: 0.4447566743654381


It gives you a score between 0 and 1 based on the grade of similarity and this grade of similarity signifies how much two sets are overlapping each other


Cosine distance is smaller than Jaccard distance.